## 1. Softmax classifier (like logistic)

In [1]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [22]:
training_epoch = 25
learning_rate = 0.01
batch_size = 100

# Graph Input
X = tf.placeholder("float", [None, 784]) # 784개의 feature (28 x 28)
Y = tf.placeholder("float", [None, 10]) # 1 부터 10

# Set model weight
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [23]:
# Model
activation = tf.nn.softmax(tf.matmul(X, W) + b)

# Cost function (Cross entropy : 실제값(Y)과 log예측값(activation)의 곱을 합한 후 평균을 냄) 
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(activation), reduction_indices=1))

# optimization
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [24]:
# Initialization
init = tf.global_variables_initializer()

In [25]:
# Launch
with tf.Session() as sess:
    sess.run(init)
    
    # Training
    for epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size) # 55000개 training data
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(100) # 1회에 100개씩 550번 loop
            _, c = sess.run([optimizer, cost], feed_dict={X : batch_xs, Y : batch_ys})
            
            avg_cost += c / total_batch
            
        if epoch % 1 == 0:
            print "Epoch: ", "%04d" % (epoch + 1), "cost=", "{:.9f}".format(avg_cost)
        
    print "Optimization Finished"
    
    # Test
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(Y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy: ", accuracy.eval({X : mnist.test.images, Y : mnist.test.labels})

Epoch:  0001 cost= 1.185057776
Epoch:  0002 cost= 0.664978850
Epoch:  0003 cost= 0.552604859
Epoch:  0004 cost= 0.499552738
Epoch:  0005 cost= 0.465079360
Epoch:  0006 cost= 0.442369879
Epoch:  0007 cost= 0.425785417
Epoch:  0008 cost= 0.411793717
Epoch:  0009 cost= 0.402091410
Epoch:  0010 cost= 0.391296188
Epoch:  0011 cost= 0.385442796
Epoch:  0012 cost= 0.377984825
Epoch:  0013 cost= 0.372287661
Epoch:  0014 cost= 0.367212594
Epoch:  0015 cost= 0.362602765
Epoch:  0016 cost= 0.358736135
Epoch:  0017 cost= 0.354563447
Epoch:  0018 cost= 0.351663312
Epoch:  0019 cost= 0.348035226
Epoch:  0020 cost= 0.345456941
Epoch:  0021 cost= 0.342695041
Epoch:  0022 cost= 0.341055348
Epoch:  0023 cost= 0.337562731
Epoch:  0024 cost= 0.335623243
Epoch:  0025 cost= 0.333540952
Optimization Finished
Accuracy:  0.9135


In [19]:
mnist.test.images.shape

(10000, 784)

In [14]:
mnist.train.next_batch(10)

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]]))

## 2. Neural Network

In [26]:
# Parameter
learning_rate = 0.001
training_epoch = 25
batch_size = 100

In [27]:
# Graph input
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

# Layer and bias
W1 = tf.Variable(tf.random_normal([784, 256]))
W2 = tf.Variable(tf.random_normal([256, 256]))
W3 = tf.Variable(tf.random_normal([256, 10]))

b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([10]))

In [29]:
# Model
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
hypothesis = tf.add(tf.matmul(L2, W3), b3)

In [30]:
# cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [33]:
# Initialization
init = tf.global_variables_initializer()

In [34]:
# Launch
with tf.Session() as sess:
    sess.run(init)
    
    # Training
    for epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size) # 55000개 training data
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(100) # 1회에 100개씩 550번 loop
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={X : batch_xs, Y : batch_ys})
            # average loss
            avg_cost += c / total_batch
            
        if epoch % 1 == 0:
            print "Epoch: ", "%04d" % (epoch + 1), "cost=", "{:.9f}".format(avg_cost)
        
    print "Optimization Finished"
    
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({X : mnist.test.images, Y : mnist.test.labels})

Epoch:  0001 cost= 190.389436864
Epoch:  0002 cost= 42.198810425
Epoch:  0003 cost= 26.685083371
Epoch:  0004 cost= 18.405697818
Epoch:  0005 cost= 13.401345751
Epoch:  0006 cost= 9.991915153
Epoch:  0007 cost= 7.508097746
Epoch:  0008 cost= 5.585075273
Epoch:  0009 cost= 4.168195659
Epoch:  0010 cost= 3.183739734
Epoch:  0011 cost= 2.362985620
Epoch:  0012 cost= 1.798196025
Epoch:  0013 cost= 1.357462142
Epoch:  0014 cost= 1.064521588
Epoch:  0015 cost= 0.867181389
Epoch:  0016 cost= 0.654093022
Epoch:  0017 cost= 0.543513392
Epoch:  0018 cost= 0.542527065
Epoch:  0019 cost= 0.530578956
Epoch:  0020 cost= 0.459948440
Epoch:  0021 cost= 0.352804120
Epoch:  0022 cost= 0.259783851
Epoch:  0023 cost= 0.382489484
Epoch:  0024 cost= 0.309532927
Epoch:  0025 cost= 0.336732629
Optimization Finished
Accuracy: 0.9548


## 3. NN with xavier initialization (초기 가중치 설정)

In [35]:
def xavier_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [36]:
# Parameter
learning_rate = 0.001
training_epoch = 25
batch_size = 100

In [37]:
# Graph input
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

# Layer and bias
W1 = tf.get_variable("W1", shape = [784, 256], initializer=xavier_init(784, 256))
W2 = tf.get_variable("W2", shape = [256, 256], initializer=xavier_init(256, 256))
W3 = tf.get_variable("W3", shape = [256, 10], initializer=xavier_init(256, 10))

b1 = tf.Variable(tf.zeros([256]))
b2 = tf.Variable(tf.zeros([256]))
b3 = tf.Variable(tf.zeros([10]))

In [38]:
# Model
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
hypothesis = tf.add(tf.matmul(L2, W3), b3)

In [39]:
# cost and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [49]:
# Initialization
init = tf.global_variables_initializer()

In [41]:
# Launch
with tf.Session() as sess:
    sess.run(init)
    
    # Training
    for epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size) # 55000개 training data
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(100) # 1회에 100개씩 550번 loop
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={X : batch_xs, Y : batch_ys})
            # average loss
            avg_cost += c / total_batch
            
        if epoch % 1 == 0:
            print "Epoch: ", "%04d" % (epoch + 1), "cost=", "{:.9f}".format(avg_cost)
        
    print "Optimization Finished"
    
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({X : mnist.test.images, Y : mnist.test.labels})

Epoch:  0001 cost= 0.302407114
Epoch:  0002 cost= 0.113428493
Epoch:  0003 cost= 0.076841262
Epoch:  0004 cost= 0.053509536
Epoch:  0005 cost= 0.039907459
Epoch:  0006 cost= 0.030712249
Epoch:  0007 cost= 0.024475898
Epoch:  0008 cost= 0.020319543
Epoch:  0009 cost= 0.016330425
Epoch:  0010 cost= 0.012699563
Epoch:  0011 cost= 0.013715739
Epoch:  0012 cost= 0.009314998
Epoch:  0013 cost= 0.011381943
Epoch:  0014 cost= 0.011401653
Epoch:  0015 cost= 0.008828260
Epoch:  0016 cost= 0.008933662
Epoch:  0017 cost= 0.008461802
Epoch:  0018 cost= 0.007110051
Epoch:  0019 cost= 0.005589009
Epoch:  0020 cost= 0.009963807
Epoch:  0021 cost= 0.004982289
Epoch:  0022 cost= 0.007009625
Epoch:  0023 cost= 0.010225282
Epoch:  0024 cost= 0.005035763
Epoch:  0025 cost= 0.004123504
Optimization Finished
Accuracy: 0.9766


## 4. More deep and dropout

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
# Parameter
learning_rate = 0.001
training_epoch = 25
batch_size = 100
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def xavier_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [4]:
# Graph input
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])
dropout_rate = tf.placeholder(tf.float32)

# Layer and bias
W1 = tf.get_variable("W1", shape=[784, 500], initializer=xavier_init(784,500))
W2 = tf.get_variable("W2", shape=[500, 256], initializer=xavier_init(500,256))
W3 = tf.get_variable("W3", shape=[256, 128], initializer=xavier_init(256,128))
W4 = tf.get_variable("W4", shape=[128, 64], initializer=xavier_init(128,64))
W5 = tf.get_variable("W5", shape=[64, 10], initializer=xavier_init(64,10))


b1 = tf.Variable(tf.zeros([500]))
b2 = tf.Variable(tf.zeros([256]))
b3 = tf.Variable(tf.zeros([128]))
b4 = tf.Variable(tf.zeros([64]))
b5 = tf.Variable(tf.zeros([10]))

In [6]:
# Construct model
dropout_rate = tf.placeholder("float")
_L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))
L1 = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
L2 = tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))
L3 = tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.add(tf.matmul(L3, W4), b4))
L4 = tf.nn.dropout(_L4, dropout_rate)

hypothesis = tf.add(tf.matmul(L4, W5), b5)

In [7]:
# cost and optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [10]:
# Initialize
init = tf.global_variables_initializer()

In [11]:
# Launch
with tf.Session() as sess:
    sess.run(init)
    
    # Training
    for epoch in range(training_epoch):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size) # 55000개 training data
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(100) # 1회에 100개씩 550번 loop
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={X : batch_xs, 
                                                          Y : batch_ys,
                                                         dropout_rate:0.7})
            # average loss
            avg_cost += c / total_batch
            
        if epoch % 1 == 0:
            print "Epoch: ", "%04d" % (epoch + 1), "cost=", "{:.9f}".format(avg_cost)
        
    print "Optimization Finished"
    
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({X : mnist.test.images, 
                                      Y : mnist.test.labels, 
                                     dropout_rate: 1})

Epoch:  0001 cost= 0.475071080
Epoch:  0002 cost= 0.190262621
Epoch:  0003 cost= 0.143219065
Epoch:  0004 cost= 0.115039476
Epoch:  0005 cost= 0.098969297
Epoch:  0006 cost= 0.088314197
Epoch:  0007 cost= 0.079146385
Epoch:  0008 cost= 0.072226720
Epoch:  0009 cost= 0.067553020
Epoch:  0010 cost= 0.059198628
Epoch:  0011 cost= 0.058165367
Epoch:  0012 cost= 0.050675838
Epoch:  0013 cost= 0.050999129
Epoch:  0014 cost= 0.047629396
Epoch:  0015 cost= 0.044065304
Epoch:  0016 cost= 0.041274765
Epoch:  0017 cost= 0.039294836
Epoch:  0018 cost= 0.041203228
Epoch:  0019 cost= 0.036919394
Epoch:  0020 cost= 0.037396075
Epoch:  0021 cost= 0.033318758
Epoch:  0022 cost= 0.036256944
Epoch:  0023 cost= 0.032199785
Epoch:  0024 cost= 0.029453857
Epoch:  0025 cost= 0.032543999
Optimization Finished
Accuracy: 0.9816


In [15]:
tf.zeros([784,10])

<tf.Tensor 'zeros_27:0' shape=(784, 10) dtype=float32>